
# Test twitter data from github repo

Load data from github repo

In [2]:
import pandas as pd
import os

import altair as alt

# Load Corona data


In [3]:
covid_data = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv")

covid_data["date"] = pd.to_datetime(covid_data["date"])

covid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50513 entries, 0 to 50512
Data columns (total 41 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   iso_code                         50222 non-null  object        
 1   continent                        49931 non-null  object        
 2   location                         50513 non-null  object        
 3   date                             50513 non-null  datetime64[ns]
 4   total_cases                      46865 non-null  float64       
 5   new_cases                        49650 non-null  float64       
 6   new_cases_smoothed               48867 non-null  float64       
 7   total_deaths                     38215 non-null  float64       
 8   new_deaths                       49650 non-null  float64       
 9   new_deaths_smoothed              48867 non-null  float64       
 10  total_cases_per_million          46610 non-null  float64  

In [ ]:
covid_data["date"] = pd.to_datetime(covid_data["date"])

cov_selection = covid_data[["date", "total_cases", "new_cases", "new_deaths"]]

cov_selection = cov_selection.groupby('date').sum().reset_index()

start_date = '12-31-2019'
end_date = '04-07-2020'

mask = (cov_selection['date'] > start_date) & (cov_selection['date'] <= end_date)

cov_selection = cov_selection.loc[mask]

cov_selection.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98 entries, 1 to 98
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         98 non-null     datetime64[ns]
 1   total_cases  98 non-null     float64       
 2   new_cases    98 non-null     float64       
 3   new_deaths   98 non-null     float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 3.8 KB


# Load news articles from github project

Load CNN articles from xlsx file

In [ ]:
cnn_news = pd.read_excel("https://github.com/xxz-jessica/COVID-19_UCD_Challenge/blob/master/News_Fox_CNN/foxnews_article_form.xlsx?raw=true")
cnn_news = cnn_news.dropna()

cnn_news = cnn_news.rename(columns = {"true_time": "published_date"})
cnn_news["published_date"] = pd.to_datetime(cnn_news["published_date"])

counted_news_cnn = cnn_news["published_date"].value_counts()
counted_news_cnn = counted_news_cnn.to_frame()
mask2 = (counted_news_cnn.index > start_date) & (counted_news_cnn.index <= end_date)
counted_news_cnn = counted_news_cnn.loc[mask2]

counted_news_cnn.reset_index(level=0, inplace=True)
counted_news_cnn = counted_news_cnn.rename(columns = {"index" : "date", "published_date" : "cnn_articles"})


counted_news_cnn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          84 non-null     datetime64[ns]
 1   cnn_articles  84 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 1.4 KB


In [ ]:
fox_news = pd.read_excel("https://github.com/xxz-jessica/COVID-19_UCD_Challenge/blob/master/News_Fox_CNN/foxnews_article_form.xlsx?raw=true")

fox_news = fox_news.dropna()

fox_news = fox_news.rename(columns = {"true_time": "published_date"})

fox_news["published_date"] = pd.to_datetime(fox_news["published_date"])

counted_news_fox = fox_news["published_date"].value_counts()
counted_news_fox = counted_news_fox.to_frame()
mask3 = (counted_news_fox.index > start_date) & (counted_news_fox.index <= end_date)
counted_news_fox = counted_news_fox.loc[mask3]

counted_news_fox.reset_index(level=0, inplace=True)
counted_news_fox = counted_news_fox.rename(columns = {"index" : "date", "published_date" : "fox_articles"})


counted_news_fox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          84 non-null     datetime64[ns]
 1   fox_articles  84 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 1.4 KB


# Visualize number of daily tweets, news articles and Covid cases from github project

In [ ]:
num_tweets = pd.read_csv("https://raw.githubusercontent.com/xxz-jessica/COVID-19_UCD_Challenge/master/Tweet_Raw_Data/tweet_per_day_1.csv")

num_tweets["Tweets"] = num_tweets["Tweets"].str.replace(",", "").astype(int)
num_tweets["Day"] = pd.to_datetime(num_tweets["Day"])

num_tweets = num_tweets.rename(columns = {"Day": "date"})

num_tweets.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    98 non-null     datetime64[ns]
 1   Tweets  98 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 1.7 KB


In [ ]:
#merge data
merged_3 = pd.merge(num_tweets, cov_selection).merge(counted_news_cnn, how = "outer").merge(counted_news_fox, how = "outer")
merged_3.info()

merged_3 = merged_3.fillna(0)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98 entries, 0 to 97
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          98 non-null     datetime64[ns]
 1   Tweets        98 non-null     int64         
 2   total_cases   98 non-null     float64       
 3   new_cases     98 non-null     float64       
 4   new_deaths    98 non-null     float64       
 5   cnn_articles  84 non-null     float64       
 6   fox_articles  84 non-null     float64       
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 6.1 KB


In [ ]:
base = alt.Chart(merged_3).encode(
    alt.X('date:T', axis=alt.Axis(title=None))
).properties(
    width=500,
    height=200
)


line_tweets = base.mark_line(stroke='#1da1f2', interpolate='monotone').encode(
    alt.Y('Tweets',
          axis=alt.Axis(title='Number of tweets', titleColor='#1da1f2'))
)

line_cases = base.mark_line( interpolate='monotone', stroke='#eb34eb').encode(
    alt.Y('new_cases',
          axis=alt.Axis(title='New cases', titleColor='#eb34eb'))
)

alt.layer(line_tweets, line_cases).resolve_scale(
    y = 'independent'
)

alt.LayerChart(...)

In [ ]:
columns = ['new_cases', 'new_deaths']

select_box = alt.binding_select(options=columns, name='Pandemic data')
sel = alt.selection_single(fields=['column'], bind=select_box, init={'column': 'new_cases'})



news_chart = alt.Chart(merged_3).transform_fold(
    columns,
    as_=['column', 'value']
).transform_filter(
    sel  
).mark_line(interpolate='monotone', stroke='#fc0303').encode(
    x='date',
    y='value:Q',
).add_selection(
    sel
).properties(
    width=500,
    height=200
)

line_tweets = base.mark_line(stroke='#1da1f2', interpolate='monotone').encode(
    alt.Y('Tweets',
          axis=alt.Axis(title='Number of tweets', titleColor='#1da1f2'))
)

alt.layer(news_chart, line_tweets).resolve_scale(
    y = 'independent'
)

alt.LayerChart(...)

# Compare tweet numbers to news articles on CNN and Fox News

In [ ]:
columns = ['fox_articles', 'cnn_articles']

select_box = alt.binding_select(options=columns, name='News source')
sel = alt.selection_single(fields=['column'], bind=select_box, init={'column': 'cnn_articles'})



news_chart = alt.Chart(merged_3).transform_fold(
    columns,
    as_=['column', 'value']
).transform_filter(
    sel  
).mark_line(interpolate='monotone', stroke='#fc0303').encode(
    x='date',
    y='value:Q',
).add_selection(
    sel
).properties(
    width=500,
    height=200
)

line_tweets = base.mark_line(stroke='#1da1f2', interpolate='monotone').encode(
    alt.Y('Tweets',
          axis=alt.Axis(title='Number of tweets', titleColor='#1da1f2'))
)

alt.layer(news_chart, line_tweets).resolve_scale(
    y = 'independent'
)

alt.LayerChart(...)

# Load downloaded twitter raw data from Github

In [ ]:
import pandas as pd
import os

#path = r'C:/Users/rapha/DATA_VIS/twitter_project/data/raw_data/'
onlinepath = "https://raw.githubusercontent.com/xxz-jessica/COVID-19_UCD_Challenge/master/Tweet_Raw_Data/raw_data/"
#files = os.listdir(path)

### filelist generated through download
files_gen = ['covid2019_0120.csv',
 'covid2019_0121.csv',
 'covid2019_0122.csv',
 'covid2019_0123.csv',
 'covid2019_0124.csv',
 'covid2019_0125.csv',
 'covid2019_0126.csv',
 'covid2019_0127.csv',
 'covid2019_0128.csv',
 'covid2019_0129.csv',
 'covid2019_0130.csv',
 'covid2019_0131.csv',
 'covid2019_0201.csv',
 'covid2019_0202.csv',
 'covid2019_0203.csv',
 'covid2019_0204.csv',
 'covid2019_0205.csv',
 'covid2019_0206.csv',
 'covid2019_0207.csv',
 'covid2019_0208.csv',
 'covid2019_0209.csv',
 'covid2019_0210.csv',
 'covid2019_0211.csv',
 'covid2019_0212.csv',
 'covid2019_0213.csv',
 'covid2019_0214.csv',
 'covid2019_0215.csv',
 'covid2019_0216.csv',
 'covid2019_0217.csv',
 'covid2019_0218.csv',
 'covid2019_0219.csv',
 'covid2019_0220.csv',
 'covid2019_0308.csv',
 'covid2019_0309.csv',
 'covid2019_0310.csv',
 'covid2019_0311.csv',
 'covid2019_0312.csv',
 'covid2019_0313.csv',
 'covid2019_0314.csv',
 'covid2019_0315.csv',
 'covid2019_0316.csv',
 'covid2019_0317.csv',
 'covid2019_0318.csv',
 'covid2019_0319.csv',
 'covid2019_0320.csv',
 'covid2019_0321.csv',
 'covid2019_0322.csv',
 'covid2019_0323.csv',
 'covid2019_0324.csv',
 'covid2019_0325.csv',
 'covid2019_0326.csv',
 'covid2019_0327.csv',
 'covid2019_0328.csv',
 'covid2019_0329.csv',
 'covid2019_0330.csv',
 'covid2019_0331.csv',
 'covid2019_0401.csv',
 'covid2019_0402.csv',
 'covid2019_0403.csv',
 'covid2019_0404.csv',
 'covid2019_0405.csv',
 'covid2019_0406.csv',
 'covid2019_0407.csv',
 'covid2019_0408.csv',
 'random tweet 2020-02-21.csv',
 'random tweet 2020-02-22.csv',
 'random tweet 2020-02-23.csv',
 'random tweet 2020-02-24.csv',
 'random tweet 2020-02-25.csv',
 'random tweet 2020-02-26.csv',
 'random tweet 2020-02-27.csv',
 'random tweet 2020-02-28.csv',
 'random tweet 2020-02-29.csv',
 'random tweet 2020-03-01.csv',
 'random tweet 2020-03-02.csv',
 'random tweet 2020-03-03.csv',
 'random tweet 2020-03-04.csv',
 'random tweet 2020-03-05.csv',
 'random tweet 2020-03-06.csv',
 'random tweet 2020-03-07.csv']

In [ ]:
covid_twitter_data = pd.DataFrame()
# Concat the Twitters data into one-table
for file in files_gen:
    data = pd.read_csv(str(path) + file)
    covid_twitter_data = covid_twitter_data.append(data, ignore_index=True)

NameError: ignored

In [ ]:
#covid_twitter_data.info()

#covid_sample = pd.read_csv("https://raw.githubusercontent.com/xxz-jessica/COVID-19_UCD_Challenge/master/Tweet_Raw_Data/raw_data/covid2019_0120.csv")

covid_twitter_data.info()

# Find keywords in the data

keyword_list: global list of keywords we want to search for in the data

count_keywords: function to count keywords in a tweet dataframe, best done for each day individually


In [ ]:
#use singular word (if plural just adds an "s")
#for several words use space (eg. "social distancing"; "social_distancing" and "socialdistancing" automatically searched for)
#if there are different words with the same meaning that do not contain each other, use "|" (eg. "New York|NY")
#if two words should be in there but not necessarily together, use &
keyword_list = ["trump", "social distancing", "Wuhan", "chinese virus", "New York", "Los Angeles", "Chicago", "Washington", 
               "mask", "economy", "healthcare", "fauci", "death|deadly|mortality"  ]

In [ ]:
# Trump counter
import re

def count_keywords(keywords, tweetlist):
    #function takes a list of keyword and a twitter dataset and counts the number of tweets containing each keyword
    #return the keywords and the corresponding tweet counts
    #search for words is case insensitive 
    #if word contains space (social distancing) it also searches for socialdistancing and social_distancing
    date = pd.to_datetime(tweetlist["timestamp"][0]).date()
    counterdf = pd.DataFrame({'date': [date] })
    counterlist = []
    for word in keywords:
        counter = 0
        if " " in word:
            searchword = word + "|" + word.replace(" ", "_")
            searchword = searchword + "|" + word.replace(" ", "")
        else:
            searchword = word
        for tweet in tweetlist["text"].dropna():
            if re.search(searchword, tweet, re.IGNORECASE):
                counter = counter + 1
        counterdf[word] = counter

    return counterdf


In [ ]:

def count_keywords_complete(path = "https://raw.githubusercontent.com/xxz-jessica/COVID-19_UCD_Challenge/master/Tweet_Raw_Data/raw_data/", 
                            files_gen = files_gen,
                            keywords = keyword_list):
    #create dataframe that gives number of tweets containing hashtag for all dates for which data is available
    countdata = pd.DataFrame()
    for file in files_gen:
        data = pd.read_csv(path + file)
        countdata = countdata.append(count_keywords(keywords, data))
    countdata.sort_values(by = "date")
    return countdata
    
df = count_keywords_complete(files_gen = files_gen)

InvalidURL: ignored

In [ ]:
df

# Load data of trending topics from github

selecting the fifty most talked about trending topics on twitter

manually annotate the wider topic the trend belongs to (C: Coronavirus, P: Politics, D: Tweets related to a special day of the week/year, O: Other topics)



In [ ]:
#trending = pd.read_csv("trending_topics_usa.csv")

trending = pd.read_csv("https://raw.githubusercontent.com/xxz-jessica/COVID-19_UCD_Challenge/master/Twitter_Trending_Topics/usa_trending.csv", encoding='latin1')
trending["date"] = pd.to_datetime(trending["date"])


trending_counted = trending.groupby('trending')['date'].apply(list).reset_index(name='dates')
trending_counted2 = trending.groupby("trending").count().reset_index()

#trending_counted = trending_counted.sort_values("num_dates", ascending = False)

#trending_counted.merge(trending_counted2, left_on =)

trending_selected = trending_counted.merge(trending_counted2).sort_values("date", ascending = False)

trending_selected = trending_selected[0:50]

trending_selected["annotation"] = ["P", "C", "O", "O", "C", "P", "P", "C", "P", "P", "D", "D", "O", "D", "D", 
                                    "D", "O", "D", "O", "P", "D", "C", "P", "D", "C", "P", "C", "P", "O", "O",
                                    "P", "P", "O", "C", "O", "O", "O", "D", "P", "O", "P", "P", "D", "C", "P", 
                                    "P", "P", "P", "D", "P"]


# Create wordcloud using brunel

In [ ]:
import brunel

In [ ]:
trending_select
%brunel cloud data("trending_selected") size(date) label(trending) tooltip(dates) color(annotation)